In [4]:
!pip install openpyxl

In [7]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import glob

def read_and_merge_excels(folder_path):
    # Get all Excel file paths in the folder
    excel_files = glob.glob(f"{folder_path}/*.xlsx")  # Adjust for .xls if needed

    # List to store DataFrames
    dataframes = []

    # Read each Excel file
    for file in excel_files:
        df = pd.read_excel(file, engine="openpyxl")  # Use engine="xlrd" for .xls files
        dataframes.append(df)

    # Concatenate all DataFrames
    merged_df = pd.concat(dataframes, ignore_index=True)

    return merged_df

# Example usage
folder_path = "/content/drive/My Drive/AssessmentData"  # Update with your folder path
final_df = read_and_merge_excels(folder_path)

# Display first few rows
print(final_df.head())

               OrderId     OrderItemId  QuantityOrdered  ItemPrice  \
0  171-0001135-1657958  11168926687715                1      949.0   
1  171-0001497-9165123  19760298917699                1      699.0   
2  171-0002127-1363507   5949764099083                1      399.0   
3  171-0002370-0601169  57571868836379                1      499.0   
4  171-0004526-2028348  33851287891403                1     1699.0   

                            PromotionDiscount  batch_id  
0    { "CurrencyCode": "INR", "Amount": "10"}       359  
1  { "CurrencyCode": "INR", "Amount": "10.1"}      1135  
2    { "CurrencyCode": "INR", "Amount": "10"}       297  
3  { "CurrencyCode": "INR", "Amount": "10.1"}       114  
4    { "CurrencyCode": "INR", "Amount": "10"}       764  


In [10]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PandasToSpark").getOrCreate()

In [12]:
# 1. Combine the data from both regions into a single table.
df = spark.createDataFrame(final_df)
df.show(truncate=False)

+-------------------+--------------+---------------+---------+------------------------------------------+--------+
|OrderId            |OrderItemId   |QuantityOrdered|ItemPrice|PromotionDiscount                         |batch_id|
+-------------------+--------------+---------------+---------+------------------------------------------+--------+
|171-0001135-1657958|11168926687715|1              |949.0    |{ "CurrencyCode": "INR", "Amount": "10"}  |359     |
|171-0001497-9165123|19760298917699|1              |699.0    |{ "CurrencyCode": "INR", "Amount": "10.1"}|1135    |
|171-0002127-1363507|5949764099083 |1              |399.0    |{ "CurrencyCode": "INR", "Amount": "10"}  |297     |
|171-0002370-0601169|57571868836379|1              |499.0    |{ "CurrencyCode": "INR", "Amount": "10.1"}|114     |
|171-0004526-2028348|33851287891403|1              |1699.0   |{ "CurrencyCode": "INR", "Amount": "10"}  |764     |
|171-0004781-3853173|43686103544491|1              |399.0    |{ "CurrencyCode": 

In [22]:
# 2. Add a column total_sales which is calculated as QuantityOrdered * ItemPrice.

from pyspark.sql.functions import col

df = df.withColumn('total_sales',col("QuantityOrdered")*col("ItemPrice") )
df.show(truncate=False)
df.count()

+-------------------+--------------+---------------+---------+------------------------------------------+--------+-----------+
|OrderId            |OrderItemId   |QuantityOrdered|ItemPrice|PromotionDiscount                         |batch_id|total_sales|
+-------------------+--------------+---------------+---------+------------------------------------------+--------+-----------+
|171-0001135-1657958|11168926687715|1              |949.0    |{ "CurrencyCode": "INR", "Amount": "10"}  |359     |949.0      |
|171-0001497-9165123|19760298917699|1              |699.0    |{ "CurrencyCode": "INR", "Amount": "10.1"}|1135    |699.0      |
|171-0002127-1363507|5949764099083 |1              |399.0    |{ "CurrencyCode": "INR", "Amount": "10"}  |297     |399.0      |
|171-0002370-0601169|57571868836379|1              |499.0    |{ "CurrencyCode": "INR", "Amount": "10.1"}|114     |499.0      |
|171-0004526-2028348|33851287891403|1              |1699.0   |{ "CurrencyCode": "INR", "Amount": "10"}  |764   

88988

In [23]:
# 3. Add a column region to identify the region of the sales record (A or B).


In [26]:
# 4. Ensure that there are no duplicate entries based on OrderId.
df = df.dropDuplicates(['OrderId'])
df.show(truncate=False)
df.count()

+-------------------+--------------+---------------+---------+------------------------------------------+--------+-----------+
|OrderId            |OrderItemId   |QuantityOrdered|ItemPrice|PromotionDiscount                         |batch_id|total_sales|
+-------------------+--------------+---------------+---------+------------------------------------------+--------+-----------+
|171-0001135-1657958|11168926687715|1              |949.0    |{ "CurrencyCode": "INR", "Amount": "10"}  |359     |949.0      |
|171-0002127-1363507|5949764099083 |1              |399.0    |{ "CurrencyCode": "INR", "Amount": "10"}  |297     |399.0      |
|171-0004781-3853173|43686103544491|1              |399.0    |{ "CurrencyCode": "INR", "Amount": "10.1"}|809     |399.0      |
|171-0004947-4305927|15941372058555|1              |1399.0   |{ "CurrencyCode": "INR", "Amount": "10"}  |15      |1399.0     |
|171-0006030-2254725|31456208605443|1              |499.0    |{ "CurrencyCode": "INR", "Amount": "10.1"}|1494  

41107

In [27]:
df.printSchema()

root
 |-- OrderId: string (nullable = true)
 |-- OrderItemId: long (nullable = true)
 |-- QuantityOrdered: long (nullable = true)
 |-- ItemPrice: double (nullable = true)
 |-- PromotionDiscount: string (nullable = true)
 |-- batch_id: long (nullable = true)
 |-- total_sales: double (nullable = true)



In [29]:
# 5. Add a new column net_sale, calculated as total_sales - PromotionDiscount.

from pyspark.sql.functions import get_json_object
df = df.withColumn("PromotionDiscountValue", get_json_object(col("PromotionDiscount"), "$.Amount").cast("double"))

df = df.withColumn("net_sale", col("total_sales") - col("PromotionDiscountValue"))
df.show(truncate=False)
df.count()

+-------------------+--------------+---------------+---------+------------------------------------------+--------+-----------+----------------------+--------+
|OrderId            |OrderItemId   |QuantityOrdered|ItemPrice|PromotionDiscount                         |batch_id|total_sales|PromotionDiscountValue|net_sale|
+-------------------+--------------+---------------+---------+------------------------------------------+--------+-----------+----------------------+--------+
|171-0001135-1657958|11168926687715|1              |949.0    |{ "CurrencyCode": "INR", "Amount": "10"}  |359     |949.0      |10.0                  |939.0   |
|171-0002127-1363507|5949764099083 |1              |399.0    |{ "CurrencyCode": "INR", "Amount": "10"}  |297     |399.0      |10.0                  |389.0   |
|171-0004781-3853173|43686103544491|1              |399.0    |{ "CurrencyCode": "INR", "Amount": "10.1"}|809     |399.0      |10.1                  |388.9   |
|171-0004947-4305927|15941372058555|1         

41107

In [32]:
# 6. Exclude orders where the total sales amount is negative or zero after applying discounts.

df = df.filter(col("net_sale") > 0)
df.show()

+-------------------+--------------+---------------+---------+--------------------+--------+-----------+----------------------+--------+
|            OrderId|   OrderItemId|QuantityOrdered|ItemPrice|   PromotionDiscount|batch_id|total_sales|PromotionDiscountValue|net_sale|
+-------------------+--------------+---------------+---------+--------------------+--------+-----------+----------------------+--------+
|171-0001135-1657958|11168926687715|              1|    949.0|{ "CurrencyCode":...|     359|      949.0|                  10.0|   939.0|
|171-0002127-1363507| 5949764099083|              1|    399.0|{ "CurrencyCode":...|     297|      399.0|                  10.0|   389.0|
|171-0004781-3853173|43686103544491|              1|    399.0|{ "CurrencyCode":...|     809|      399.0|                  10.1|   388.9|
|171-0004947-4305927|15941372058555|              1|   1399.0|{ "CurrencyCode":...|      15|     1399.0|                  10.0|  1389.0|
|171-0006030-2254725|31456208605443|     

In [34]:
# 7. Load the transformed data into a the database of your choice.
import sqlite3
conn = sqlite3.connect("orders.db")

In [36]:
pdf = df.toPandas()
pdf.to_sql("orders", conn, if_exists="replace", index=False)
conn.close()